In [4]:
import pyautogui
import shutil
import time
from PIL import ImageGrab
from PIL import Image
import os
import keyboard
import pyperclip
import wx
import lxk
import threading

flag = 0
def stop():
    keyboard.wait('f2')
    flag=1
    
t=threading.Thread(target=stop)
t.start()

image_path = 'D:/AutoClickPicture/'
cache = ''
try:
    os.mkdir(image_path)
except:
    pass
def click_image(image, confidence=0.95, lag_x=0,lag_y=0,wait=0.2, drag=None, double=False,loop=False):
    while True:

        # 获取屏幕分辨率
        screen_width, screen_height = pyautogui.size()

        # 全屏截图
        screen_image = ImageGrab.grab()

        # 在屏幕截图中查找匹配
        location = pyautogui.locate(image, screen_image, confidence=confidence)
        if location:
            time.sleep(wait)
            location = pyautogui.locate(image, screen_image, confidence=confidence)
            # 匹配成功，返回位置信息
            x, y, a, b = location
            if drag==None:
                if double==False:
                    pyautogui.click(x+a//2+lag_x, y+b//2+lag_y)
                else:
                    pyautogui.doubleClick(x+a//2+lag_x, y+b//2+lag_y)
            else:
                pyautogui.moveTo(x+a//2+lag_x, y+b//2+lag_y)
                pyautogui.mouseDown()
                pyautogui.moveRel(drag[0],drag[1],0.2)
                pyautogui.mouseUp()
            return True
        else:
            # 匹配失败，返回None
            if loop==False or flag==1:return False
            return
            time.sleep(10)

In [2]:
class btnFrame(wx.Frame):
    def __init__(self, parent, btn_group):        
        self.flag = -1
        self.temp=''
        self.pic_group =[]
        w =250
        h =len(btn_group)*30+150
        wx.Frame.__init__(self, parent, title='My Frame', size=(w, h))
        
        
        sizer = wx.BoxSizer(wx.VERTICAL)
        self.text_ctrl = wx.TextCtrl(self, style=wx.TE_PROCESS_ENTER)
        self.text_ctrl.SetHint("功能文本框") # 设置默认提示文字
        if cache !='':self.text_ctrl.SetValue(cache)
        sizer.Add(self.text_ctrl, 0, wx.ALL, 5)
        for i in btn_group:
            exec(f"self.{i}=wx.Button(self, label='{btn_group[i]}')") # 创建btn
            exec(f"sizer.Add(self.{i}, 0, wx.ALL, 5)") # 添加进
            exec(f"self.{i}.Bind(wx.EVT_BUTTON, self.On_{i})")

        self.SetSizer(sizer)
        
        # 初始化窗口
        self.Show()
        self.SetWindowStyle(wx.DEFAULT_FRAME_STYLE | wx.STAY_ON_TOP)
        
    def On_toggle_btn(self, event):
        if self.GetWindowStyle() & wx.STAY_ON_TOP:
            self.SetWindowStyle(wx.DEFAULT_FRAME_STYLE)
            self.text_ctrl.SetValue('取消置顶')
        else:
            self.SetWindowStyle(wx.DEFAULT_FRAME_STYLE | wx.STAY_ON_TOP)
            self.text_ctrl.SetValue('已置顶')
            
    def On_add_btn(self, event):
        self.flag +=1
        self.pic_group.insert(self.flag,ImageGrab.grabclipboard())
        if click_image(self.pic_group[self.flag]):
            self.text_ctrl.SetValue('点击成功')
        else:
            self.text_ctrl.SetValue('点击失败')
    
    def On_debug_btn(self, event):
        x,y=pyautogui.position()
        if self.flag==len(self.pic_group)-1:return
        self.flag+=1
        if click_image(self.pic_group[self.flag]):
            self.text_ctrl.SetValue('点击成功')
        else:
            self.text_ctrl.SetValue('点击失败')  
        pyautogui.moveTo(x,y)
        
    def On_reset_btn(self, event):
        self.flag=-1
    
    def On_save_btn(self, event):
        global cache
        self.temp = self.text_ctrl.GetValue()
        cache = self.temp
        try:
            os.mkdir(image_path+f'{self.temp}/')
        except:
            shutil.rmtree(image_path+f'{self.temp}/')
            os.mkdir(image_path+f'{self.temp}/')
        for i in range(0,len(self.pic_group)):
            self.pic_group[i].save(image_path+f'{self.temp}/'+f'{i}.png')
        
    def On_load_btn(self, event):
        global cache
        cache = self.temp
        self.temp = self.text_ctrl.GetValue()
        self.pic_group=[]
        self.flag=-1
        for i in os.listdir(image_path+f'{self.temp}/'):
            image = Image.open(image_path+f'{self.temp}/{i}')
            self.pic_group.append(image)
            
    def On_start_btn(self, event):
        time.sleep(10)
        l=len(self.pic_group)
        for i in range(l):
            click_image(self.pic_group[i],loop=True)
            self.text_ctrl.SetValue(f'{i}执行完成')
        self.text_ctrl.SetValue('执行完成')
        
try:
    del app
except:
    pass
app = wx.App()
btn_group = {'toggle_btn':'始终置顶','add_btn':'添加',\
                     'debug_btn':'调试','reset_btn':'重置','save_btn':'保存','load_btn':'加载','start_btn':'执行'}
frame = btnFrame(None,btn_group)
app.MainLoop()

0

In [ ]:
直接改成手动运行算了

In [7]:

image = Image.open(image_path+'0.png')
while True:
    click_image(image,loop=True)

KeyboardInterrupt: 